<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PostgreSQL_04C_Ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
Tutorial based on [SQLCourse2](https://www.sqlcourse2.com/) <br>
Data is in this [Google Sheet](https://docs.google.com/spreadsheets/d/1m_LE_ahVHY0PrXmtyVVjO5CyqvmOMOwGT6hPYBYHBqw/edit?usp=sharing) <br>
The data needs to be downloaded as two CSV files from the Google Docs above

#Install & Configure Postgres

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2024-01-07 17:16:05.461934+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


For Connecting to a Postgres server on Amazon RDS, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_01_Install_Config.ipynb)

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [3]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:praxisdb:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


In [5]:
# Download two CSV files from G-Drive into the Colab VM
#!gdown https://drive.google.com/uc?id=1WxuruueFwMTjmPb-DvvVnGYB78Ty4OkP
#!gdown https://drive.google.com/uc?id=1j49sqYHv0UoskVJDzNObWxECFF7ySsld

#!gdown -qq 1Ug15GbEmeTnwJnuH-4fpPeZ1k20ryoGx      # Quiet Mode, not output
#!gdown -qq 1qx6duSqzQUJSkpHNQIIZ9l0T6zYtjT-y      # Quiet Mode, not output

!wget -O Customers.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20Customers.csv
!wget -O ItemsOrdered.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20ItemsOrdered.csv

In [6]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists itemsordered'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int, price decimal(6,2)) \
'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists customers'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS customers (customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dt+'

NOTICE:  table "itemsordered" does not exist, skipping
DROP TABLE
CREATE TABLE
NOTICE:  table "customers" does not exist, skipping
DROP TABLE
CREATE TABLE
                                       List of relations
 Schema |     Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+--------------+-------+---------+-------------+---------------+---------+-------------
 public | customers    | table | upraxis | permanent   | heap          | 0 bytes | 
 public | itemsordered | table | upraxis | permanent   | heap          | 0 bytes | 
(2 rows)



In [7]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually, password = 'pass'
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY customers FROM '/content/Customers.csv' DELIMITER ',' CSV HEADER;"
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY itemsordered FROM '/content/ItemsOrdered.csv' DELIMITER ',' CSV HEADER;"

COPY 17
COPY 32


In [9]:
#!cat ItemsOrdered.csv
#!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'select * from itemsordered'

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [10]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [11]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [13]:
#runSelect("select * from itemsordered")
runSelect("select * from customers")

customerid,firstname,lastname,city,state
10101,John,Gray,Lynden,Washington
10298,Leroy,Brown,Pinetop,Arizona
10299,Elroy,Keller,Snoqualmie,Washington
10315,Lisa,Jones,Oshkosh,Wisconsin
10325,Ginger,Schultz,Pocatello,Idaho
10329,Kelly,Mendoza,Kailua,Hawaii
10330,Shawn,Dalton,Cannon Beach,Oregon
10338,Michael,Howell,Tillamook,Oregon
10339,Anthony,Sanchez,Winslow,Arizona
10408,Elroy,Cleaver,Globe,Arizona


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-12-31 16:01:25.642107+05:30
